## Required important packages

In [20]:
# ! pip install segment-geospatial
# ! conda install -c conda-forge gdal

<module 'GeoFeatures' from 'C:\\Users\\Administrator\\Documents\\Jupyter\\Image Registration\\GeoFeatures.py'>

In [27]:
import os
# import leafmap
import torch
import rasterio
import geopandas as gpd
import numpy as np
import json
import fiona
from samgeo.hq_sam import SamGeo
import GeoUtils
import GeoFeatures
import SegmentPlots
from shapely.geometry import Point
import math
import matplotlib.pyplot as plt

## Used inputs

In [147]:
sam_model_path = os.path.join('File_path', 'SAM', 'sam_hq_vit_h.pth') # path containing SAM model weights

folderName = 'Rice'# Master folder containing the reference and sensed images
s1_FileName = '01-09-2023' # File name of image for epoch 1 (refrence image)
s1_FolderPath = os.path.join(folderName, s1_FileName)
s1_img_input = os.path.join(s1_FolderPath, s1_FileName + '.tif')

s2_FileName = '08-09-2023' # File name of image for epoch 2 (sensed image)
s2_FolderPath = os.path.join(folderName, s2_FileName)
s2_img_input = os.path.join(s2_FolderPath, s2_FileName + '.tif')

gcp_template_file_path = os.path.join(folderName, 'Results', 'points.txt') # Path for the template to store the GCPs points in QGIS compliant format
gcp_out_file = os.path.join(folderName, 'Results', s1_FileName+'-'+s2_FileName+'.points')

s1_input_raster = os.path.join(s1_FolderPath, s1_FileName + '-resampled-annotation.tif')
s1_output_vector = os.path.join(s1_FolderPath, s1_FileName + '-resampled-vector.shp')
s2_input_raster = os.path.join(s2_FolderPath, s2_FileName + '-resampled-annotation.tif')
s2_output_vector = os.path.join(s2_FolderPath, s2_FileName + '-resampled-vector.shp')

s1_vector_input_Path = os.path.join(s1_FolderPath,s1_FileName+'-resampled-vector.shp')
s2_vector_input_Path = os.path.join(s2_FolderPath,s2_FileName+'-resampled-vector.shp')

s1_centroids_transformed_path = os.path.join(s1_FolderPath,s1_FileName+'-centroids-transfromed.shp')
s2_centroids_transformed_path = os.path.join(s2_FolderPath,s2_FileName+'-centroids-transfromed.shp')

matched_features_path = os.path.join(folderName,'Results','matchedFeatures-'+s1_FileName+'-'+s2_FileName+'.json')
matched_lines_path = os.path.join(folderName,'Results','matchedLines-'+s1_FileName+'-'+s2_FileName+'.shp')
matched_lines_inliers_path = os.path.join(folderName,'Results','matchedLinesInliers-'+s1_FileName+'-'+s2_FileName+'.shp')
add_matched_lines_inliers_path = os.path.join(folderName,'Results','addMatchedLinesInliers-'+s1_FileName+'-'+s2_FileName+'.shp')
add_matched_features_path = os.path.join(folderName,'Results','addMatchedLinesInliers-'+s1_FileName+'-'+s2_FileName+'.json')

downSampleFactor = 1 # Down sample factor e.g  use values 1, 2, 3 ...
range_percentage = 0.25 # Percentage of the median of plot areas for selecting plots. It helps in removing noisy segmentations.
crs = "EPSG:4326"
min_Neighbors = 5 # minimum number of neighbors based on which distance threshold for the feature selection is calculated
neighbor_percentile = 90 # percentile of centroids which contains min number of Neighbors
gps_error = 8.0 # error in meters
gps_error_dia = 2*gps_error # max allowable distance between two corresponding points from two sessions being matched

angle_tolerance = 3 # Upper bound on angle difference beyond which match is not valid. Reduces the matched pairs
dist_tolerance = 0.075 # Upper bound on distance difference beyond which match is not valid. Reduces the matched pairs

bin_length_dist = 0.25 # defines distance window for identifying the peak from the histogram
bin_length_brg = 15 # defines angle window for identifying the peak from the histogram

## Downsample the orthomosaics (reference, sensed) for reducing segmentation computational complexity for SAM model

In [148]:
s1_img_input_resampled = GeoUtils.resampledGeoTiff(s1_img_input, downSampleFactor)
s2_img_input_resampled = GeoUtils.resampledGeoTiff(s2_img_input, downSampleFactor)
s1_img_input_reprojected = GeoUtils.change_CRS_raster(s1_img_input_resampled, crs)
s2_img_input_reprojected = GeoUtils.change_CRS_raster(s2_img_input_resampled, crs)

## Segment the plots using Segment Anything Model

In [149]:
sam_kwargs = {"points_per_side": 32, "pred_iou_thresh": 0.85, "stability_score_thresh": 0.85,
    "crop_n_layers": 0, "crop_n_points_downscale_factor": 1, "min_mask_region_area": 100}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(checkpoint=sam_model_path, model_type='vit_h', device=device, sam_kwargs=None)
SegmentPlots.generateSegmentationMask(s1_img_input_resampled, sam, crs)
SegmentPlots.generateSegmentationMask(s2_img_input_resampled, sam, crs)

Unique values saved to: Rice\08-09-2023\08-09-2023-resampled-annotation-summed.tif
Unique values saved to: Rice\22-09-2023\22-09-2023-resampled-annotation-summed.tif


# Convert the segmented polygons in the form of geo-raster to geo-spatial vector file format (shapefile)

In [ ]:
SegmentPlots.raster_to_polygon(s1_input_raster, s1_output_vector) 
SegmentPlots.raster_to_polygon(s2_input_raster, s2_output_vector)

# Identify and select the desired polygons while eliminating the noisy polygons based on the spatial dimensions

In [1]:
s1_selected_polygons = SegmentPlots.selectPolygons(s1_vector_input_Path, range_percentage)
s2_selected_polygons = SegmentPlots.selectPolygons(s2_vector_input_Path, range_percentage)

# Generate the centroids for the polygons

In [2]:
s1_centroids = SegmentPlots.generateCentroids(s1_selected_polygons, s1_vector_input_Path)
s2_centroids = SegmentPlots.generateCentroids(s2_selected_polygons, s2_vector_input_Path)

# Compute feature descriptor for reference image

In [153]:
# Identify and drop rows with duplicate values in the 'coordinate' column
s1_centroids = s1_centroids[~s1_centroids.duplicated(subset='geometry', keep='first')]
# Reproject the centroids into the preferred crs e.g "EPSG:4326"
s1_centroids_transformed = GeoUtils.transformCRS(s1_centroids, crs)
# Sort the centroids based on latitude
s1_centroids_transformed = GeoUtils.sortGeoDataframe(s1_centroids_transformed)
# Assign id to each centroid
s1_centroids_transformed['id'] = range(1, len(s1_centroids_transformed) + 1)
# Calculate radius for defining the neighborhood of centroids
dist_dict = GeoFeatures.getDistanceDictionary(s1_centroids_transformed)
distanceThresholdNeighbors = GeoFeatures.getDistThresholdNeighbors(dist_dict, min_Neighbors, neighbor_percentile)
s1_neighbor_dist = GeoFeatures.get_neighbor_distances(s1_centroids_transformed, distanceThresholdNeighbors)
# Calculate feature vector for each feature (centroid)
s1_featureDescriptor = GeoFeatures.get_feature_descriptor(s1_centroids_transformed,s1_neighbor_dist)
# Append the feature descriptor to the geopandas dataframe
s1_centroids_transformed = GeoFeatures.appendFeatureDescriptor(s1_centroids_transformed, s1_featureDescriptor) 
# Save the transformed centroid file
s1_centroids_transformed.to_file(s1_centroids_transformed_path, driver='ESRI Shapefile')

# Compute feature descriptor for sensed image

In [154]:
# Identify and drop rows with duplicate values in the 'coordinate' column
s2_centroids = s2_centroids[~s2_centroids.duplicated(subset='geometry', keep='first')]
# Reproject the centroids into the preferred crs e.g "EPSG:4326"
s2_centroids_transformed = GeoUtils.transformCRS(s2_centroids, crs)
# Sort the centroids based on latitude
s2_centroids_transformed = GeoUtils.sortGeoDataframe(s2_centroids_transformed)
# Assign id to each centroid
s2_centroids_transformed['id'] = range(1, len(s2_centroids_transformed) + 1)
s2_neighbor_dist = GeoFeatures.get_neighbor_distances(s2_centroids_transformed, distanceThresholdNeighbors)
# Calculate radius for defining the neighborhood of centroids
s2_featureDescriptor = GeoFeatures.get_feature_descriptor(s2_centroids_transformed,s2_neighbor_dist)
# Append the feature descriptor to the geopandas dataframe
s2_centroids_transformed = GeoFeatures.appendFeatureDescriptor(s2_centroids_transformed, s2_featureDescriptor) 
# Save the transformed centroid file
s2_centroids_transformed.to_file(s2_centroids_transformed_path, driver='ESRI Shapefile')

# Compute putative matches

In [3]:
matched_features = GeoFeatures.matchFeatures(s1_centroids_transformed, s2_centroids_transformed, gps_error_dia, angle_tolerance, dist_tolerance)
# Remove entries with empty list
matched_features = {key: value for key, value in matched_features.items() if value not in (None, [])}
# Save the matches as dictionary to a JSON file
with open(matched_features_path, 'w') as json_file:
    json.dump(matched_features, json_file)
    
# Save the matches from a JSON file as dictionary    
# with open(matched_features_path, 'r') as json_file:
#     matched_features = json.load(json_file)

# Draw lines between matched centroids (helpful for visualizations)
matched_List = GeoFeatures.generateSrcToDstPointsList(s1_centroids_transformed, s2_centroids_transformed, matched_features)
GeoFeatures.drawLinesMatchedPoints(matched_List, matched_lines_path, crs)

# Identify inliers and remove outliers

In [6]:
(distList, brgList) = GeoFeatures.getDistBearingList(matched_List)
# Compute distance and angle windows for identifying peaks from distList and brgList
thresholds = GeoFeatures.getDistBearingThresholds(distList, brgList, bin_length_dist, bin_length_brg)
# Remove outliers based on the geometric transform
matched_features_inliers = GeoFeatures.removeOutlierPoints(s1_centroids_transformed, s2_centroids_transformed, matched_features, thresholds)
# Identify inliers
matched_inliers_List = GeoFeatures.generateSrcToDstPointsList(s1_centroids_transformed, s2_centroids_transformed, matched_features_inliers)
# Save inliers for visualizations
GeoFeatures.drawLinesMatchedPoints(matched_inliers_List, matched_lines_inliers_path, crs)
# Identify additional matches using the geometric orientation of the inliers
add_match_thresholds = GeoFeatures.get_add_matches_theshold(matched_inliers_List)
add_matched_inliers_dict = GeoFeatures.add_feature_matches(s1_centroids_transformed, s2_centroids_transformed, add_match_thresholds)
add_matched_inliers_List = GeoFeatures.generateSrcToDstPointsList(s1_centroids_transformed, s2_centroids_transformed, add_matched_inliers_dict)
# Save final matches for visualizations
GeoFeatures.drawLinesMatchedPoints(add_matched_inliers_List, add_matched_lines_inliers_path, crs)

# Save the final matches as dictionary to a JSON file
with open(add_matched_features_path, 'w') as json_file:
    json.dump(add_matched_inliers_dict, json_file)
    
# Read the final matches from a JSON file as dictionary    
# with open(add_matched_features_path, 'r') as json_file:
#     add_matched_inliers_dict = json.load(json_file)

# Save the matched pair points in the form of ground control points (GCPs) suitable for georeferencing in QGIS
GeoUtils.write_GCP_file(gcp_template_file_path, gcp_out_file, add_matched_inliers_List)

In [7]:
print("Number of initial matches: " + str(len(matched_List)))
print("Number of inliers: " + str(len(matched_inliers_List)))
print("Number of final matches: " + str(len(add_matched_inliers_List)))